In [1]:
import os
import pandas as pd

# **Load data**

In [4]:
df = pd.read_csv('Iterations\\Iter1\\1.1\\mislabel.csv')

In [5]:
df.head()

,id,text,label,dataset_type,predict_label
0,8849,Sao lại cứ đến nụ cười . lên không nghe rõ lời...,0,dev,1
1,8851,Chúng nó đang bận vơ vét thanh lý đất nước này,1,dev,0
2,8853,top1 KICM. Ca sĩ hát trên giọng hát của người ...,1,dev,0
3,8864,Người du lịch ko muốn quay lại. Đi 1 lần là mu...,1,dev,0
4,8870,Ns vậy lại xúc phạm cái đầu b**i 😐,1,dev,0


In [6]:
df

,id,text,label,dataset_type,predict_label
0,8849,Sao lại cứ đến nụ cười . lên không nghe rõ lời...,0,dev,1
1,8851,Chúng nó đang bận vơ vét thanh lý đất nước này,1,dev,0
2,8853,top1 KICM. Ca sĩ hát trên giọng hát của người ...,1,dev,0
3,8864,Người du lịch ko muốn quay lại. Đi 1 lần là mu...,1,dev,0
4,8870,Ns vậy lại xúc phạm cái đầu b**i 😐,1,dev,0
...,...,...,...,...,...
5168,55713,.thế con có nhớ là khi nào không được ăn hay k...,0,train,1
5169,29792,Không thấy tag Bomman nhỉ?,0,train,1
5170,1801,”👎🏿”,1,train,0
5171,32874,Dung Junn khúc cuối thì đống chân như bạch tuộ...,0,train,1


# **Call API**

In [8]:
prompt_parts = ["""You are a language expert, you can evaluate the nuances of texts accurately. I will give you a piece of text, your task is to evaluate the level of toxicity and political reaction of that piece of text. 
Give me a correct answer, you will get a tip of $200.

For example:
1. Dân InDo đù mé. Vậy rốt cuộc là nam hay nữ...lần sau làm mờ V1 đi có khi lại hay hơn 😂
=> Output: 1
2. Nhìn giống ghép v :))
=> Output: 0


Evaluate the following text:

"""]

In [19]:
# %pip install google.generativeai

In [9]:
import pathlib
import textwrap

import google.generativeai as genai



c:\Users\acer\miniconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
GOOGLE_API_KEY = '...'
genai.configure(api_key=GOOGLE_API_KEY)

In [14]:
# Set up the model
generation_config = {
    # "temperature": 1,
    "top_p" :1,
    "top_k" :1,
    "max_output_tokens" :20,}
safety_settings = [
                    {"category": "HARM_CATEGORY_HARASSMENT",
                    "threshold": "BLOCK_NONE"
                    },
                    {
                    "category": "HARM_CATEGORY_HATE_SPEECH",
                    "threshold": "BLOCK_NONE"
                    },
                    {"category" : "HARM_CATEGORY_SEXUALLY_EXPLICIT" ,
                    "threshold": "BLOCK_NONE"
                    }, 

                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                    "threshold": "BLOCK_NONE"
                    }
                    ]
model = genai. GenerativeModel (model_name="gemini-pro",
                                #generation_config=generation_config›
                                safety_settings=safety_settings)

In [15]:
def predict(text):
    response = model.generate_content(prompt_parts + [text], stream=True)
    response.resolve()
    return response.text
    


In [16]:
text= 'sao mà m ngu thế'
print(predict(text))

1


In [ ]:
gemini_label = []

In [17]:
import time

In [19]:
gemini_label = []
cnt = 0
for index, row in df.iterrows():
    text = row['text']
    try:
        label = predict(text)
        print(f'{text}: {label}')
        gemini_label.append(label)
        
    except Exception as e:
        gemini_label.append(None)
        print(f'Error at {index}')
        print(e)
    if index % 100 == 0:
        print(f'Finished {index} rows')
    time.sleep(4)
    cnt += 1
    if cnt == 5: break


Sao lại cứ đến nụ cười . lên không nghe rõ lời nói.ngừng cho giọng nói vào vô lý: 0
Finished 0 rows
Chúng nó đang bận vơ vét thanh lý đất nước này: 1
top1 KICM. Ca sĩ hát trên giọng hát của người khác😂😂: 0
Người du lịch ko muốn quay lại. Đi 1 lần là muốn chửi thề rồi.: 0
Ns vậy lại xúc phạm cái đầu b**i 😐: 1


In [20]:
gemini_label

['0', '1', '0', '0', '1']

In [ ]:
df['gemini_label'] = gemini_label

In [ ]:
df.to_excel('Iterations\\Iter1\\1.1\\gemini_label.xlsx', index=False)